In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#initial load kc_table_id = "opendatadbt.311.kc311"

kc311 = requests.get("https://data.kcmo.org/resource/7at3-sxhp.json? limit=10000& where=creation_date >= '2021-03-17T00:00:00.000'" ) kc311json = kc311.json() kc_df = pd.DataFrame(kc311json) kc_df = kc_df[['creation_date', 'closed_date','department','work_group','request_type','zip_code','street_address','status','ycoordinate','xcoordinate']] kc_df

In [3]:
#Incremental load
nyc_table_id = "opendatadbt.311.kc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(creation_date), max(closed_date)
FROM `opendatadbt.311.kc311`
limit 10
"""

kcmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
kcmaxopendatedate = str(kcmaxopendatedataframe['f0_'][0])

kcmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
kcmaxclosedate = str(kcmaxclosedatedataframe['f1_'][0])


In [4]:

kc311 = requests.get("https://data.kcmo.org/resource/7at3-sxhp.json?$limit=50000&$where=creation_date>'"+kcmaxopendatedate+"' OR closed_date between '" +kcmaxclosedate+ "' AND '" + current_datetime + "'") 
kc311json = kc311.json()


In [5]:
cols = ['case_id','creation_date', 'closed_date','department','work_group','request_type','zip_code','street_address','status','ycoordinate','xcoordinate']
kc_df = pd.DataFrame(kc311json, columns=cols)
kc_df

,case_id,creation_date,closed_date,department,work_group,request_type,zip_code,street_address,status,ycoordinate,xcoordinate
0,2019040174,2019-02-25T00:00:00.000,2021-03-26T00:00:00.000,NHS,NHS-Neighborhood Preservation-,Property Violations,64132,1813 E 82nd St,RESOL,38.978267,-94.566291
1,2020150497,2020-11-30T00:00:00.000,2021-03-26T00:00:00.000,Public Works,Public Works-Capital Projects-Signs,Signs-Traffic Sign-Damaged – Non-Emergency,64116,1401 NE Parvin Rd,RESOL,39.167562,-94.563798
2,2021020211,2021-02-25T00:00:00.000,2021-03-26T00:00:00.000,Public Works,Public Works-Street and Traffic-District 1,Streets / Roadways / Alleys-Pothole-District 1,64118,401 NE Barry Rd,RESOL,39.245791,-94.575211
3,2021019737,2021-02-25T00:00:00.000,2021-03-26T00:00:00.000,Public Works,Public Works-Street and Traffic-District 2,Streets / Roadways / Alleys-Sinkhole Referral-...,64106,516 Grand Blvd,RESOL,39.108121,-94.580587
4,2021020035,2021-02-25T00:00:00.000,2021-03-26T00:00:00.000,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,64114,8500 Ward Pkwy,RESOL,38.974213,-94.607259
5,2018085134,2018-06-28T00:00:00.000,2021-03-26T00:00:00.000,NHS,NHS-Neighborhood Preservation-,Property Violations,64127,2530 Bales Ave,RESOL,39.079365,-94.542335
6,2021023905,2021-03-18T00:00:00.000,2021-03-26T00:00:00.000,NHS,NHS-Neighborhood Preservation-,Property Violations,64126,1802 Fuller Ave,RESOL,39.088268,-94.507346
7,2021023958,2021-03-19T00:00:00.000,2021-03-26T00:00:00.000,NHS,NHS-Neighborhood Preservation-Open Entry,Prop/Build/Construct-Dangerous Building-Open t...,64126,1628 Belmont Ave,RESOL,39.09094,-94.510614
8,2021024067,2021-03-22T00:00:00.000,2021-03-26T00:00:00.000,NHS,NHS-Neighborhood Preservation-,Property Violations,64132,7324 Agnes Ave,RESOL,38.993361,-94.55283
9,2021020687,2021-02-26T00:00:00.000,2021-03-26T00:00:00.000,NHS,NHS-Neighborhood Preservation-,Property Violations,64134,11506 Palmer Ave,RESOL,38.914092,-94.502781


In [6]:
kc_table_id = "opendatadbt.311.kc311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('kc311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    kc_df, table_ref, job_config=job_config
).result()  # Make an API request.
